In [1]:
#importing libraries
import pandas as pd
import numpy as np
import csv

In [2]:
df=pd.read_csv('train.csv',quoting=1, error_bad_lines=False)

In [3]:
df.head() #datset visualization

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna() #dropping the nan values

In [5]:
X = df.drop('label', axis = 1)

In [6]:
y = df['label']

In [7]:
X.shape #shape of input 

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:

tf.__version__

'2.2.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
messages = X.copy() #making copy of input

In [13]:
messages.reset_index(inplace=True)

In [14]:
import re
import nltk
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [17]:
#data cleaning
for i in range(len(messages)):
  review = re.sub('[^A-Za-z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
vocab_size = 1000

In [20]:
onehot_rep = [one_hot(word,vocab_size) for word in corpus]

In [21]:
onehot_rep

[[434, 843, 245, 958, 581, 928, 813, 180, 275, 616],
 [841, 969, 211, 89, 829, 156, 674],
 [359, 547, 230, 885],
 [497, 104, 225, 776, 923, 986],
 [63, 829, 131, 161, 500, 42, 829, 249, 170, 2],
 [424, 959, 450, 136, 941, 687, 339, 596, 715, 63, 740, 820, 479, 705, 674],
 [442, 442, 120, 901, 416, 231, 681, 184, 478, 64, 646],
 [174, 619, 783, 45, 782, 503, 687, 195, 478, 64, 646],
 [429, 979, 974, 341, 591, 626, 909, 613, 687, 263],
 [123, 789, 487, 636, 309, 239, 421, 290],
 [345, 287, 770, 241, 803, 479, 323, 846, 974, 63, 967],
 [776, 169, 581, 626, 687, 782],
 [470, 666, 238, 212, 629, 458, 62, 713, 678],
 [33, 105, 61, 317, 913, 614, 249, 478, 64, 646],
 [178, 869, 608, 463, 143, 478, 64, 646],
 [768, 42, 404, 383, 365, 531, 387, 262, 118, 293],
 [548, 953, 969],
 [838, 279, 980, 754, 687, 480, 334, 674],
 [658, 167, 211, 68, 612, 619, 93, 348, 42],
 [794, 886, 687, 277, 942, 674],
 [236, 285, 805, 169, 83, 93, 992, 243, 895, 478, 64, 646],
 [740, 304, 917, 82, 37, 203, 562],
 [4

In [22]:
#padding the sentences for the embedding layer
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre',maxlen = sent_length)
print(embedded_docs)

[[  0   0   0 ... 180 275 616]
 [  0   0   0 ... 829 156 674]
 [  0   0   0 ... 547 230 885]
 ...
 [  0   0   0 ... 478  64 646]
 [  0   0   0 ...  44 890 386]
 [  0   0   0 ... 142 892 300]]


In [23]:
len(embedded_docs)

18285

In [24]:
embedded_docs[10]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0, 345, 287, 770, 241,
       803, 479, 323, 846, 974,  63, 967], dtype=int32)

In [25]:
#creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length= sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            40000     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 96,501
Trainable params: 96,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=0)

In [29]:
#training the model
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 7s 30ms/step - loss: 0.3308 - accuracy: 0.8441 - val_loss: 0.2146 - val_accuracy: 0.9073
Epoch 2/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1905 - accuracy: 0.9199 - val_loss: 0.2097 - val_accuracy: 0.9100
Epoch 3/10
229/229 [==============================] - 6s 27ms/step - loss: 0.1701 - accuracy: 0.9292 - val_loss: 0.2064 - val_accuracy: 0.9114
Epoch 4/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1500 - accuracy: 0.9412 - val_loss: 0.2070 - val_accuracy: 0.9139
Epoch 5/10
229/229 [==============================] - 6s 27ms/step - loss: 0.1322 - accuracy: 0.9476 - val_loss: 0.2219 - val_accuracy: 0.9109
Epoch 6/10
229/229 [==============================] - 6s 27ms/step - loss: 0.1200 - accuracy: 0.9526 - val_loss: 0.2338 - val_accuracy: 0.8994
Epoch 7/10
229/229 [==============================] - 6s 27ms/step - loss: 0.1069 - accuracy: 0.9594 - val_loss: 0.2424 - val_accuracy: 0.9106

In [30]:
#prediction
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test, y_pred)

array([[1864,  176],
       [ 192, 1425]])

In [33]:
#accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.89937106918239

In [34]:
#adding dropout layers
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [35]:
#training the model
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 7s 30ms/step - loss: 0.3441 - accuracy: 0.8337 - val_loss: 0.2215 - val_accuracy: 0.9016
Epoch 2/10
229/229 [==============================] - 7s 29ms/step - loss: 0.1994 - accuracy: 0.9167 - val_loss: 0.2250 - val_accuracy: 0.9078
Epoch 3/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1823 - accuracy: 0.9258 - val_loss: 0.2092 - val_accuracy: 0.9114
Epoch 4/10
229/229 [==============================] - 7s 28ms/step - loss: 0.1661 - accuracy: 0.9325 - val_loss: 0.2134 - val_accuracy: 0.9169
Epoch 5/10
229/229 [==============================] - 7s 29ms/step - loss: 0.1536 - accuracy: 0.9373 - val_loss: 0.2053 - val_accuracy: 0.9128
Epoch 6/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1412 - accuracy: 0.9439 - val_loss: 0.2141 - val_accuracy: 0.9133
Epoch 7/10
229/229 [==============================] - 7s 30ms/step - loss: 0.1308 - accuracy: 0.9465 - val_loss: 0.2342 - val_accuracy: 0.9155

In [36]:
#predicting the output
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1876,  164],
       [ 188, 1429]])

In [38]:
#accuracy calaculation
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9037462400875034